## Model of Lupus Nephritis
#### see paper: [here](https://tbiomed.biomedcentral.com/articles/10.1186/1742-4682-7-14)

In [ ]:
import numpy as np

In [2]:

# Define Variables and parameters
Lupusvars = list(var("I", "A", "P", "D"))
sa = 0.05
si = 0.001 #rate that immune complexes deposit in the kidneys
sid = 0.002 #immune response to accumulation of damaged cells
kip = 1 # rate of immune complex removal from system
kpp = 0.02 
kpi = 0.13 #rate of mediator activation and recruitment
kpd = 0.001 #rate of activation for pro-inflammatory agents as a result of cytokine release or induced by damaged tissue
kdip = 0.025
kdp = 0.27
kap = 0.022 #level of inflammation???
kad = 0.22 #level of damage???
mup = 0.06 #decay of pro-inflammatory mediators
mua = 2.2 #rate of anti-inflammatory agent degradation
mud = 0.04 #decay rate of damage
kid = 1
Ainf = 0.45

(xmin, xmax, ymin, ymax) = (0, 1, 0, 10000)

def f(x):
    return x/((1+A/Ainf)^2)

Idot = f(si) + f(sid)*(D^2/(kid^2 + D^2)) - kip*f(P)*I
Pdot = f(kpi*I + kpp*P) + f(kpd*D) - mup*P
Ddot = kdip*f(P)*I + kdp*f(P) - mud*D
Adot = sa + f(kap*P + kad*D) - mua*A

LupusSystem = (Idot, Pdot, Ddot, Adot)
init = (0.1, 0.4, 1.7, 0.1)

tmax=60
t_range = srange(0, tmax, 1)

LupusSim = desolve_odeint(LupusSystem, init, t_range, Lupusvars)
LupusSim = np.insert(LupusSim, 0, t_range, axis=1)
Its = list_plot(LupusSim[::,(0,1)], plotjoined=True, color="grey", legend_label="I")
Pts = list_plot(LupusSim[::,(0,2)], plotjoined=True, color="blue", legend_label="P")
Dts = list_plot(LupusSim[::,(0,3)], plotjoined=True, color="red", legend_label="D")
Ats = list_plot(LupusSim[::,(0,4)], plotjoined=True, color="black", legend_label="A")
show(Its+Pts+Dts+Ats, title = "Lupus", ymax = ymax)

1
2
3
4
5
